##### Copyright 2020 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Ejemplo de entrenamiento con reconocimiento de la cuantización en Keras

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/model_optimization/guide/quantization/training_example"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver en TensorFlow.org</a>
</td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/es-419/model_optimization/guide/quantization/training_example.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Ejecutar en Google Colab</a>
</td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/es-419/model_optimization/guide/quantization/training_example.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver código fuente en GitHub</a>
</td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/es-419/model_optimization/guide/quantization/training_example.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Descargar el bloc de notas</a>
</td>
</table>

## Descripción general

Le presentamos un ejemplo de principio a fin de *entrenamiento con reconocimiento de la cuantización*.

### Otras paginas

Para obtener una introducción a qué es el entrenamiento con reconocimiento de la cuantización y determinar si debe usarlo (incluido lo que se admite), consulte la [página de descripción general](https://www.tensorflow.org/model_optimization/guide/quantization/training.md).

Para encontrar rápidamente las API que necesita para su caso de uso (más allá de cuantizar completamente un modelo con 8 bits), consulte la [guía completa](https://www.tensorflow.org/model_optimization/guide/quantization/training_comprehensive_guide.md).

### Resumen

En este tutorial, podrá:

1. Entrenar un modelo `tf.keras` para MNIST desde cero.
2. Ajustar el modelo aplicando la API de entrenamiento con reconocimiento de la cuantización, ver la precisión y exportar un modelo con reconocimiento de la cuantización.
3. Usar el modelo para crear un modelo realmente cuantizado para el backend de TFLite.
4. Ver la persistencia de la precisión en TFLite y un modelo 4 veces más pequeño. Para ver los beneficios de latencia en dispositivos móviles, pruebe los ejemplos de TFLite [en el repositorio de aplicaciones de TFLite](https://www.tensorflow.org/lite/models).

## Preparación

In [ ]:
! pip install -q tensorflow
! pip install -q tensorflow-model-optimization


In [ ]:
import tempfile
import os

import tensorflow as tf

from tensorflow import keras

## Entrenar un modelo para MNIST sin entrenamiento con reconocimiento de la cuantización

In [ ]:
# Load MNIST dataset
mnist = keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Normalize the input image so that each pixel value is between 0 to 1.
train_images = train_images / 255.0
test_images = test_images / 255.0

# Define the model architecture.
model = keras.Sequential([
  keras.layers.InputLayer(input_shape=(28, 28)),
  keras.layers.Reshape(target_shape=(28, 28, 1)),
  keras.layers.Conv2D(filters=12, kernel_size=(3, 3), activation='relu'),
  keras.layers.MaxPooling2D(pool_size=(2, 2)),
  keras.layers.Flatten(),
  keras.layers.Dense(10)
])

# Train the digit classification model
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.fit(
  train_images,
  train_labels,
  epochs=1,
  validation_split=0.1,
)

## Clonar y ajustar el modelo preentrenado con entrenamiento con reconocimiento de la cuantización


### Definir el modelo

Aplicará el entrenamiento con reconocimiento de la cuantización a todo el modelo y lo verá en el resumen del modelo. Todas las capas ahora tienen el prefijo "quant".

Tenga en cuenta que el modelo resultante reconoce la cuantización pero no está cuantizado (por ejemplo, los pesos son float32 en lugar de int8). En las siguientes secciones se muestra cómo crear un modelo cuantizado a partir del modelo con reconocimiento de la cuantización.

En la [guía completa](https://www.tensorflow.org/model_optimization/guide/quantization/training_comprehensive_guide.md), puede ver cómo cuantizar algunas capas para mejorar la precisión del modelo.

In [ ]:
import tensorflow_model_optimization as tfmot

quantize_model = tfmot.quantization.keras.quantize_model

# q_aware stands for for quantization aware.
q_aware_model = quantize_model(model)

# `quantize_model` requires a recompile.
q_aware_model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

q_aware_model.summary()

### Entrenar y evalar el modelo con respecto al punto de referencia

Para demostrar el ajuste después de entrenar el modelo solo durante una época, ajuste con un entrenamiento con reconocimiento de la cuantización un subconjunto de los datos de entrenamiento.

In [ ]:
train_images_subset = train_images[0:1000] # out of 60000
train_labels_subset = train_labels[0:1000]

q_aware_model.fit(train_images_subset, train_labels_subset,
                  batch_size=500, epochs=1, validation_split=0.1)

Para este ejemplo, hay una pérdida mínima en la precisión de la prueba después de la cuantización, en comparación con la línea de base.

In [ ]:
_, baseline_model_accuracy = model.evaluate(
    test_images, test_labels, verbose=0)

_, q_aware_model_accuracy = q_aware_model.evaluate(
   test_images, test_labels, verbose=0)

print('Baseline test accuracy:', baseline_model_accuracy)
print('Quant test accuracy:', q_aware_model_accuracy)

## Crear un modelo cuantizado para el backend de TFLite

Después de esto, tiene un modelo realmente cuantizado con pesos int8 y activaciones uint8.

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(q_aware_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]

quantized_tflite_model = converter.convert()

## Ver la persistencia de la precisión desde TF a TFLite

Defina una función ayudante para evaluar el modelo de TFLite en el conjunto de datos de prueba.

In [ ]:
import numpy as np

def evaluate_model(interpreter):
  input_index = interpreter.get_input_details()[0]["index"]
  output_index = interpreter.get_output_details()[0]["index"]

  # Run predictions on every image in the "test" dataset.
  prediction_digits = []
  for i, test_image in enumerate(test_images):
    if i % 1000 == 0:
      print('Evaluated on {n} results so far.'.format(n=i))
    # Pre-processing: add batch dimension and convert to float32 to match with
    # the model's input data format.
    test_image = np.expand_dims(test_image, axis=0).astype(np.float32)
    interpreter.set_tensor(input_index, test_image)

    # Run inference.
    interpreter.invoke()

    # Post-processing: remove batch dimension and find the digit with highest
    # probability.
    output = interpreter.tensor(output_index)
    digit = np.argmax(output()[0])
    prediction_digits.append(digit)

  print('\n')
  # Compare prediction results with ground truth labels to calculate accuracy.
  prediction_digits = np.array(prediction_digits)
  accuracy = (prediction_digits == test_labels).mean()
  return accuracy

Evalúe el modelo cuantizado y verá que la precisión de TensorFlow persiste en el backend de TFLite.

In [ ]:
interpreter = tf.lite.Interpreter(model_content=quantized_tflite_model)
interpreter.allocate_tensors()

test_accuracy = evaluate_model(interpreter)

print('Quant TFLite test_accuracy:', test_accuracy)
print('Quant TF test accuracy:', q_aware_model_accuracy)

## Ver un modelo 4 veces más pequeño a partir de la cuantización

Cree un modelo flotante de TFLite y luego verá que el modelo de TFLite cuantizado es 4 veces más pequeño.

In [ ]:
# Create float TFLite model.
float_converter = tf.lite.TFLiteConverter.from_keras_model(model)
float_tflite_model = float_converter.convert()

# Measure sizes of models.
_, float_file = tempfile.mkstemp('.tflite')
_, quant_file = tempfile.mkstemp('.tflite')

with open(quant_file, 'wb') as f:
  f.write(quantized_tflite_model)

with open(float_file, 'wb') as f:
  f.write(float_tflite_model)

print("Float model in Mb:", os.path.getsize(float_file) / float(2**20))
print("Quantized model in Mb:", os.path.getsize(quant_file) / float(2**20))

## Conclusión

En este tutorial, aprendió cómo crear modelos con reconocimiento de la cuantización con la API del kit de herramientas de optimización de modelos de TensorFlow y luego cuantizó modelos para el backend de TFLite.

Observó un beneficio de comprimir el modelo a un tamaño 4 veces más pequeño para un modelo para MNIST, con una diferencia de precisión mínima. Para ver los beneficios de latencia en dispositivos móviles, pruebe los ejemplos de TFLite [en el repositorio de aplicaciones de TFLite](https://www.tensorflow.org/lite/models).

Le recomendamos que pruebe esta nueva capacidad, que puede ser particularmente importante para la implementación en entornos con recursos limitados.
